# サーバ証明書の更新

---

構築したCoursewareHubのサーバ証明書を更新します。

## 概要


### グループ名

更新対象となるCoursewareHubを構築したときに指定したansibleのグループ名を指定します。

ansibleのグループ名を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

上のセルの出力結果を参考にしてグループ名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

### チェック

指定されたグループ名が前提条件を満たしていることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

CoursewareHubのサービスが実行されていることを確認します。

In [ ]:
cmd = "docker stack services --format '{{.Replicas}}' coursewarehub | grep '1/1' | wc -l"
!ansible {target_hub} -m shell -a "{{% raw %}} [ \$({cmd}) -eq 4 ] {{% endraw %}}"

group_varsから現在の設定値を読み込みます。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

## 証明書の更新

auth-proxyなどのコンテナで使用するサーバ証明書、秘密鍵を更新します。

### 証明書の配置 

auth-proxyコンテナで使用するサーバ証明書、秘密鍵などのファイルを配置します。出どころなどの情報を必要以上に残さないためにNotebookからの操作ではなく、ターミナルなどから **managerノードに ssh でログインして操作を行ってください**。

manager ノードのIPアドレスを確認します。表示されたIPアドレスに対して、ユーザ名`vcp`と「VCノード作成」のNotebookで設定したSSHの秘密鍵を指定することで manager ノードにsshでログインできます。

In [ ]:
!ansible {target_hub} -c local -m debug -a 'var=servicenet_ip'

サーバ証明書、秘密鍵をそれぞれ以下のセルの実行結果に表示されるパスに配置してください。

In [ ]:
!ansible {target_hub} -c local -m debug -a 'msg="/home/{{{{ansible_user}}}}/certs/{{{{master_fqdn}}}}.cer"'

In [ ]:
!ansible {target_hub} -c local -m debug -a 'msg="/home/{{{{ansible_user}}}}/certs/{{{{master_fqdn}}}}.key"'

中間CA証明書がある場合は、サーバ証明書と同じディレクトリに配置してください。中間CA証明書を配置した場合は次のセルにそのファイル名を指定してください。

In [ ]:
# (例)
# intermediate_certfile = 'nii-odca4g7rsa.cer'

intermediate_certfile = 

また個別の中間CA証明書を配置するのではなく、中間CA証明書を連結したファイルを配置することもできます。その場合は次のセルの実行結果に表示されるパスに配置してください。中間CA証明書を連結したファイルが存在している場合は、個別の中間CA証明書を指定しても**連結したファイルの方が優先**されます。

In [ ]:
!ansible {target_hub} -c local -m debug -a 'msg="/home/{{{{ansible_user}}}}/certs/{{{{master_fqdn}}}}.chained.cer"'

**証明書などの配置を行った後に、これ以降の操作を行ってください。**

証明書が配置されたことを確認します。managerノードに配置されたサーバ証明書の内容を表示してみます。

In [ ]:
!ansible {target_hub} -a \
    'openssl x509 -noout -text -in certs/{{{{master_fqdn}}}}.cer'

秘密鍵の内容を表示してみます。

In [ ]:
!ansible {target_hub} -m shell -a \
    'openssl ec -noout -text -in  certs/{{{{master_fqdn}}}}.key || \
    openssl rsa -noout -text -in  certs/{{{{master_fqdn}}}}.key'

中間CA証明書を指定した場合は、そのファイルがmanagerノードに配置されていることを確認します。また、中間CA証明書のパスをgroup_varsに保存します。

In [ ]:
if "intermediate_certfile" in vars():
    !ansible {target_hub} -a \
        'openssl x509 -noout -text -in certs/{intermediate_certfile}'
    import yaml
    from pathlib import Path
    gvars_path = Path(f"group_vars/{ugroup_name}")
    with gvars_path.open() as f:
        gvars = yaml.safe_load(f)
    gvars["intermediate_certfile"] = intermediate_certfile
    with gvars_path.open(mode='w') as f:
        yaml.safe_dump(gvars, stream=f)

中間CA証明書を連結したファイルを配置した場合は、その内容を確認します。

In [ ]:
!ansible {target_hub} -m shell -a 'chdir=certs \
    test ! -f {{{{master_fqdn}}}}.chained.cer || \
    ( openssl crl2pkcs7 -nocrl -certfile {{{{master_fqdn}}}}.chained.cer | openssl pkcs7 -print_certs -noout )'

### サーバ証明書の更新

managerノードに配置したサーバ証明書などを構築したCoursewareHub環境の所定のディレクトリに配置します。

ここで実行するplaybookが配置するファイル、ディレクトリを以下の表に示します。

|パス|用途|
|:---|:---|
|/etc/jupyterhub/nginx/certs/auth-proxy.chained.cer|nginxで利用するサーバ証明書|
|/etc/jupyterhub/nginx/certs/auth-proxy.key|nginxで利用する秘密鍵|
|/etc/jupyterhub/simplesamlphp/cert/auth-proxy.cer|simplesamlphpで利用する証明書|
|/etc/jupyterhub/simplesamlphp/cert/auth-proxy.key|simplesamlphpで利用する秘密鍵|


まず、実際に設定を変更する前にドライラン（チェックモード）でansibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-certs.yml || true

実際にサーバ証明書ファイルなどの更新を行います。

In [ ]:
!ansible-playbook -v -l {target_hub} playbooks/setup-certs.yml

配置したファイルを確認します。

In [ ]:
!ansible {target_hub} -a 'ls -l {{{{jupyterhub_cfg_dir}}}}/nginx/certs {{{{jupyterhub_cfg_dir}}}}/simplesamlphp/cert'

## コンテナの更新

コンテナを起動しなおして更新したサーバ証明書を反映させます。

auth-proxyコンテナを再起動します。

In [ ]:
!ansible {target_hub} -a 'docker service update --force coursewarehub_auth-proxy'

registryコンテナを再起動します。

In [ ]:
!ansible {target_hub} -a 'docker service update --force coursewarehub_registry'

auth-proxyコンテナのログを表示してみます。

In [ ]:
!ansible {target_hub} -a 'docker service logs -n 100 coursewarehub_auth-proxy'

registryコンテナのログを表示してみます。

In [ ]:
!ansible {target_hub} -a 'docker service logs -n 100 coursewarehub_registry'

## CoursewareHubにアクセスする

証明書を更新したCoursewareHub環境にアクセスして正しく動作していることを確認してください。

次のセルを実行するとCoursewareHubのアドレスを表示します。

In [ ]:
print(f'https://{gvars["master_fqdn"]}')